In [5]:
"""
Calculates the average distance between nearest points
From the geolocated AFD and the non-geolocated
Author: maxwell.cook@colorado.edu
"""

import os, sys, time
import pandas as pd
import geopandas as gpd
import numpy as np

from scipy.spatial import cKDTree

import warnings
warnings.filterwarnings("ignore")

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready !")

Ready !


In [9]:
# Load the VIIRS active fire data
geo_fp = os.path.join(projdir,'data/spatial/mod/VIIRS/vnp14img_geo_srm_pix_latlon_aspenfires.gpkg')
viirs_geo = gpd.read_file(geo_fp) # the geolocated data
fire_ids = list(viirs_geo['Fire_ID'].unique())
print(len(viirs_geo))

ngeo_fp = os.path.join(projdir,'data/spatial/mod/FIRMS/SNPP/SNPP-afd_aspen-fires_2018_to_2023.gpkg')
viirs_ngeo = gpd.read_file(ngeo_fp)
viirs_ngeo = viirs_ngeo[(viirs_ngeo['na_l3name'] == 'Southern Rockies') & (viirs_ngeo['NIFC_ID'].isin(fire_ids))]
print(len(viirs_ngeo))

64143
64963


In [ ]:
# Set up the cKDTree between points

In [15]:
from pyproj import Transformer

# calculate the average distance in meters
transformer = Transformer.from_crs("EPSG:4326", "EPSG:32613", always_xy=True)  
# Transform lat/lon to UTM (x/y in meters)
geo_coords = np.array(transformer.transform(
    viirs_geo['longitude'].values, viirs_geo['latitude'].values
)).T
ngeo_coords = np.array(transformer.transform(
    viirs_ngeo['LONGITUDE'].values, viirs_ngeo['LATITUDE'].values
)).T

tree = cKDTree(geo_coords)
distances, _ = tree.query(ngeo_coords, k=1)
distances.mean()

61.40914252939028